In [1]:
import yfinance as yf
import pandas as pd
import math
import numpy as np
import matplotlib.pyplot as plt
from termcolor import colored as cl

In [2]:
aapl = yf.download("AAPL", start="2015-01-01", end="2020-01-01")

/home/jupyter/.cache/pypoetry/virtualenvs/compare-multiple-backtests-2J6ybkfi-py3.10/lib/python3.10/site-packages/yfinance/utils.py:775: FutureWarning: The 'unit' keyword in TimedeltaIndex construction is deprecated and will be removed in a future version. Use pd.to_timedelta instead.
  df.index += _pd.TimedeltaIndex(dst_error_hours, 'h')
[*********************100%%**********************]  1 of 1 completed


In [3]:
def sma(data, lookback):
    sma = data.rolling(lookback).mean()
    return sma


def get_bb(data, lookback):
    std = data.rolling(lookback).std()
    upper_bb = sma(data, lookback) + std * 2
    lower_bb = sma(data, lookback) - std * 2
    middle_bb = sma(data, lookback)
    return upper_bb, lower_bb, middle_bb


aapl["upper_bb"], aapl["middle_bb"], aapl["lower_bb"] = get_bb(aapl["Close"], 20)
aapl = aapl.dropna()
aapl.tail()

,Open,High,Low,Close,Adj Close,Volume,upper_bb,middle_bb,lower_bb
Date,,,,,,,,,
2019-12-24,71.172501,71.222504,70.730003,71.067497,69.147995,48478800,71.863032,64.175466,68.019249
2019-12-26,71.205002,72.495003,71.175003,72.477501,70.519920,93121200,72.550129,64.128870,68.339499
2019-12-27,72.779999,73.492500,72.029999,72.449997,70.493156,146266000,73.149260,64.078738,68.613999
2019-12-30,72.364998,73.172501,71.305000,72.879997,70.911568,144114400,73.747399,64.087349,68.917374
2019-12-31,72.482498,73.419998,72.379997,73.412498,71.429665,100805600,74.312720,64.259278,69.285999


In [4]:
def get_stoch_osc(high, low, close, k_lookback, d_lookback):
    lowest_low = low.rolling(k_lookback).min()
    highest_high = high.rolling(k_lookback).max()
    k_line = ((close - lowest_low) / (highest_high - lowest_low)) * 100
    d_line = k_line.rolling(d_lookback).mean()
    return k_line, d_line


aapl["%k"], aapl["%d"] = get_stoch_osc(aapl["High"], aapl["Low"], aapl["Close"], 14, 3)
aapl.tail()

/var/tmp/ipykernel_564750/1436118945.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  aapl['%k'], aapl['%d'] = get_stoch_osc(aapl['High'], aapl['Low'], aapl['Close'], 14, 3)
/var/tmp/ipykernel_564750/1436118945.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  aapl['%k'], aapl['%d'] = get_stoch_osc(aapl['High'], aapl['Low'], aapl['Close'], 14, 3)


,Open,High,Low,Close,Adj Close,Volume,upper_bb,middle_bb,lower_bb,%k,%d
Date,,,,,,,,,,,
2019-12-24,71.172501,71.222504,70.730003,71.067497,69.147995,48478800,71.863032,64.175466,68.019249,97.202051,94.654622
2019-12-26,71.205002,72.495003,71.175003,72.477501,70.519920,93121200,72.550129,64.128870,68.339499,99.720753,98.620711
2019-12-27,72.779999,73.492500,72.029999,72.449997,70.493156,146266000,73.149260,64.078738,68.613999,85.650331,94.191045
2019-12-30,72.364998,73.172501,71.305000,72.879997,70.911568,144114400,73.747399,64.087349,68.917374,91.284202,92.218429
2019-12-31,72.482498,73.419998,72.379997,73.412498,71.429665,100805600,74.312720,64.259278,69.285999,98.799222,91.911252


In [5]:
def bb_stoch_strategy(prices, k, d, upper_bb, lower_bb):
    buy_price = []
    sell_price = []
    bb_stoch_signal = []
    signal = 0

    for i in range(len(prices)):
        if (
            k[i - 1] > 30
            and d[i - 1] > 30
            and k[i] < 30
            and d[i] < 30
            and prices[i] < lower_bb[i]
        ):
            if signal != 1:
                buy_price.append(prices[i])
                sell_price.append(np.nan)
                signal = 1
                bb_stoch_signal.append(signal)
            else:
                buy_price.append(np.nan)
                sell_price.append(np.nan)
                bb_stoch_signal.append(0)
        elif (
            k[i - 1] < 70
            and d[i - 1] < 70
            and k[i] > 70
            and d[i] > 70
            and prices[i] > upper_bb[i]
        ):
            if signal != -1:
                buy_price.append(np.nan)
                sell_price.append(prices[i])
                signal = -1
                bb_stoch_signal.append(signal)
            else:
                buy_price.append(np.nan)
                sell_price.append(np.nan)
                bb_stoch_signal.append(0)
        else:
            buy_price.append(np.nan)
            sell_price.append(np.nan)
            bb_stoch_signal.append(0)

    sell_price[-1] = prices[-1]
    bb_stoch_signal[-1] = -1
    return buy_price, sell_price, bb_stoch_signal


buy_price, sell_price, bb_stoch_signal = bb_stoch_strategy(
    aapl["Close"], aapl["%k"], aapl["%d"], aapl["upper_bb"], aapl["lower_bb"]
)

/var/tmp/ipykernel_564750/488504868.py:8: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  if k[i-1] > 30 and d[i-1] > 30 and k[i] < 30 and d[i] < 30 and prices[i] < lower_bb[i]:
/var/tmp/ipykernel_564750/488504868.py:18: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  elif k[i-1] < 70 and d[i-1] < 70 and k[i] > 70 and d[i] > 70 and prices[i] > upper_bb[i]:
/var/tmp/ipykernel_564750/488504868.py:10: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[

In [6]:
position = []
for i in range(len(bb_stoch_signal)):
    if bb_stoch_signal[i] > 1:
        position.append(0)
    else:
        position.append(1)

for i in range(len(aapl["Close"])):
    if bb_stoch_signal[i] == 1:
        position[i] = 1
    elif bb_stoch_signal[i] == -1:
        position[i] = 0
    else:
        position[i] = position[i - 1]

k = aapl["%k"]
d = aapl["%d"]
upper_bb = aapl["upper_bb"]
lower_bb = aapl["lower_bb"]
close_price = aapl["Close"]
bb_stoch_signal = (
    pd.DataFrame(bb_stoch_signal)
    .rename(columns={0: "bb_stoch_signal"})
    .set_index(aapl.index)
)
position = (
    pd.DataFrame(position)
    .rename(columns={0: "bb_stoch_position"})
    .set_index(aapl.index)
)

frames = [close_price, k, d, upper_bb, lower_bb, bb_stoch_signal, position]
strategy = pd.concat(frames, join="inner", axis=1)

strategy.tail()

,Close,%k,%d,upper_bb,lower_bb,bb_stoch_signal,bb_stoch_position
Date,,,,,,,
2019-12-24,71.067497,97.202051,94.654622,71.863032,68.019249,0,1
2019-12-26,72.477501,99.720753,98.620711,72.550129,68.339499,0,1
2019-12-27,72.449997,85.650331,94.191045,73.149260,68.613999,0,1
2019-12-30,72.879997,91.284202,92.218429,73.747399,68.917374,0,1
2019-12-31,73.412498,98.799222,91.911252,74.312720,69.285999,-1,0


In [7]:
aapl_ret = pd.DataFrame(np.diff(aapl["Close"])).rename(columns={0: "returns"})
bb_stoch_strategy_ret = []

for i in range(len(aapl_ret)):
    returns = aapl_ret["returns"][i] * strategy["bb_stoch_position"][i]
    bb_stoch_strategy_ret.append(returns)

bb_stoch_strategy_ret_df = pd.DataFrame(bb_stoch_strategy_ret).rename(
    columns={0: "bb_stoch_returns"}
)
investment_value = 100000
number_of_stocks = math.floor(investment_value / aapl["Close"][0])
bb_stoch_investment_ret = []

for i in range(len(bb_stoch_strategy_ret_df["bb_stoch_returns"])):
    returns = number_of_stocks * bb_stoch_strategy_ret_df["bb_stoch_returns"][i]
    bb_stoch_investment_ret.append(returns)

bb_stoch_investment_ret_df = pd.DataFrame(bb_stoch_investment_ret).rename(
    columns={0: "investment_returns"}
)
total_investment_ret = round(sum(bb_stoch_investment_ret_df["investment_returns"]), 2)
profit_percentage = math.floor((total_investment_ret / investment_value) * 100)
print(
    cl(
        "Profit gained from the BB STOCH strategy by investing $100k in AAPL : {}".format(
            total_investment_ret
        ),
        attrs=["bold"],
    )
)
print(
    cl(
        "Profit percentage of the BB STOCH strategy : {}%".format(profit_percentage),
        attrs=["bold"],
    )
)

Profit gained from the BB STOCH strategy by investing $100k in AAPL : 115922.36
Profit percentage of the BB STOCH strategy : 115%


/var/tmp/ipykernel_564750/4013942913.py:5: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  returns = aapl_ret['returns'][i]*strategy['bb_stoch_position'][i]
/var/tmp/ipykernel_564750/4013942913.py:10: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  number_of_stocks = math.floor(investment_value/aapl['Close'][0])
